In [1]:
# Use autoplot 207's code for this app
import datetime
import sys

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [25]:
STORM_NUMBER = 8
TITLE = "13-14 January 2025"
SUBTITLE = "8 AM 14 January 2025"
SETPOINT_LOCS = {}
sts = utc(2025, 1, 14, 12)
ets = utc(2025, 1, 15, 18)
# Get available data
ctx = {
    "coop": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
# add zeros and QC
df = add_zeros(df, ctx)

In [26]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            "Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
            "IEM 2022-2023 Winter Storm #%s"
        )
        % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [27]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,NY,BGM,67.0,-75.64000,43.32000,POINT (1934324.753 106246.024),True,0,True,LSR,55,21
1,NY,BGM,65.0,-75.62000,43.31000,POINT (1936177.466 105650.121),True,1,True,LSR,55,21
2,NY,BUF,65.0,-75.89000,43.43000,POINT (1911676.878 112122.481),True,2,True,LSR,55,21
3,NY,BGM,65.0,-75.72000,43.36000,POINT (1926917.247 108634.622),True,3,True,LSR,55,21
4,NY,BGM,60.0,-75.48000,43.42000,POINT (1943273.89 120844.887),True,4,True,LSR,56,22
...,...,...,...,...,...,...,...,...,...,...,...,...
4043,IA,DMX,0.0,-94.74910,40.98860,POINT (440631.111 -431919.122),True,CRNI4,True,COOP,6,3
4044,IA,DMX,0.0,-92.37550,42.53130,POINT (623776.138 -245411.397),True,ALXI4,True,COOP,12,9
4045,IA,DMX,0.0,-93.56110,41.96250,POINT (531893.156 -316839.824),True,AESI4,True,COOP,9,7
4046,IA,ARX,0.0,-91.53307,42.68533,POINT (690674.131 -221576.148),True,SPTI4,True,COOP,14,10


In [31]:
def main():
    setpoints = [
        (372, 0.5),
        (374, 0.5,),
        (311, 2),
        (312, 2),
        (313, 2),
    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'WSHI4',
        'GTTI4',
        'IA-CH-7',
        'POCI4',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        (236, 1),
        (251, 1),
        (157, 2.1),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=2)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250115.png")
    mp.close()


main()

     state  wfo  val       lon       lat                             geo  \
145     IA  DMX  2.1 -94.20000  42.58000   POINT (474495.18 -252254.505)   
156     IA  DMX  2.0 -94.15000  42.52000  POINT (479042.962 -258621.408)   
157     IA  DMX  2.1 -93.80000  42.47000  POINT (508048.549 -262059.082)   
196     IA  DVN  1.1 -90.54000  41.69000  POINT (783685.035 -322815.383)   
204     IA  DVN  1.0 -90.55000  41.83000  POINT (781124.958 -307421.832)   
...    ...  ...  ...       ...       ...                             ...   
1074    IA  DMX  0.0 -93.36500  40.70670  POINT (558923.859 -454801.538)   
1075    IA  ARX  0.0 -92.92350  43.38169  POINT (571056.243 -155150.279)   
1076    IA  ARX  0.0 -92.31346  43.04522  POINT (623573.727 -187982.563)   
1078    IA  ARX  0.0 -92.71764  43.28372  POINT (588571.351 -164548.533)   
1080    IA  ARX  0.0 -91.09590  42.78573  POINT (724974.993 -206672.548)   

      used_for_analysis    nwsli  plotme source  xcell  ycell  
145                True